Firstly try to get some data and show it...

In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://scrapeme.live/shop/Bulbasaur/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')


product_name = soup.find('h1', class_='product_title').text.strip()


product_price = soup.find('p', class_='price').find('span', class_='woocommerce-Price-amount amount').text.strip()

#Buradaki class bilgilerini web sitesinin source kaynağını inceleyerek çektim.
product_description = soup.find('div', class_='woocommerce-product-details__short-description').text.strip()


stock_info = soup.find('p', class_='stock').text.strip()

print("Product Information:")
print("--------------------")
print(f"Name: {product_name}")
print(f"Price: {product_price}")
print(f"Description: {product_description}")
print(f"Stock: {stock_info}")



Product Information:
--------------------
Name: Bulbasaur
Price: £63.00
Description: Bulbasaur can be seen napping in bright sunlight. There is a seed on its back. By soaking up the sun’s rays, the seed grows progressively larger.
Stock: 45 in stock


In [ ]:
import requests
from bs4 import BeautifulSoup
import json
from kafka import KafkaProducer


bootstrap_servers = 'localhost:9092'
topic_name = 'DBrain'


producer = KafkaProducer(bootstrap_servers=[bootstrap_servers],
                         value_serializer=lambda x: json.dumps(x).encode('utf-8'))


def scrape_and_send_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

  
    product_name = soup.find('h1', class_='product_title').text.strip()
    product_price = soup.find('p', class_='price').find('span', class_='woocommerce-Price-amount amount').text.strip()
    product_description = soup.find('div', class_='woocommerce-product-details__short-description').text.strip()
    stock_info = soup.find('p', class_='stock').text.strip()


    data = {
        'product_name': product_name,
        'product_price': product_price,
        'product_description': product_description,
        'stock_info': stock_info
    }


    producer.send(topic_name, value=data)
    print(f"Sent data to Kafka: {data}")

    producer.flush()

if __name__ == "__main__":
    url = 'https://scrapeme.live/shop/Bulbasaur/'
    
    scrape_and_send_data(url)


Sent data to Kafka: {'product_name': 'Bulbasaur', 'product_price': '£63.00', 'product_description': 'Bulbasaur can be seen napping in bright sunlight. There is a seed on its back. By soaking up the sun’s rays, the seed grows progressively larger.', 'stock_info': '45 in stock'}


In [38]:
import requests
from kafka import KafkaProducer
import json
import time
from bs4 import BeautifulSoup


bootstrap_servers = 'localhost:9092'

topic_name = 'DBrain'


def fetch_and_send_to_kafka():
    url = 'https://scrapeme.live/shop/'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    products = []
    for product in soup.find_all('div', class_='product'):
        name = product.find('h2').text.strip()
        price = product.find('span', class_='price').text.strip()

        products.append({'name': name, 'price': price})

    producer = KafkaProducer(bootstrap_servers=bootstrap_servers,
                             value_serializer=lambda x: json.dumps(x).encode('utf-8'))

    for item in products:
        producer.send(topic_name, value=item)
        print(f"Sent to Kafka: {item}")
        time.sleep(1)  

    producer.flush()


if __name__ == '__main__':
    fetch_and_send_to_kafka()


In [43]:
import requests
from bs4 import BeautifulSoup
import json
from kafka import KafkaProducer
import time


kafka_bootstrap_servers = 'localhost:9092'
kafka_topic = 'DBrain'

def fetch_product_data():
    url = "https://scrapeme.live/shop/"
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        products = []

    
        product_items = soup.find_all('li', class_='product')

        for item in product_items[:15]:  #buradaki 15 tane veri almak istedim.
            product = {}
            product['title'] = item.find('h2', class_='woocommerce-loop-product__title').text.strip()
            product['price'] = item.find('span', class_='price').text.strip().split()[0]  
            product['product_id'] = item.find('a', class_='button')['data-product_id']
            products.append(product)

        return products

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

def send_to_kafka(data):
    producer = KafkaProducer(bootstrap_servers=kafka_bootstrap_servers,
                             value_serializer=lambda v: json.dumps(v).encode('utf-8'))
    for product in data:
        producer.send(kafka_topic, value=product)
    producer.flush()
    print("Data sent to Kafka.")

def save_to_file(data):
    with open('products.json', 'w') as file:
        json.dump(data, file, indent=4)
    print("Data saved to products.json.")

def main():
  
    product_data = fetch_product_data()
    if not product_data:
        return
    
    
    send_to_kafka(product_data)

   
    save_to_file(product_data)

if __name__ == "__main__":
    main()


Data sent to Kafka.
Data saved to products.json.
